# Phishing Website Detection by Machine Learning - Feature Extraction

This notebook aims to collect the relevant data and extract the selective features.

In [2]:
import os
import requests
import pandas as pd
from urllib.parse import urlparse, urlencode
import ipaddress
import re
from bs4 import BeautifulSoup
import urllib
import urllib.request
from datetime import datetime

# 1. Data Collection:

For this project we are required to collect legitimate(0) and phishing URLs.

- Phishing URLs are collected using opensource service called PhishTank, http://data.phishtank.com/data/online-valid.csv.
- Legitimate URLs are collected using dataset provided by University of New Brunswick, https://www.unb.ca/cic/datasets/url-2016.html. The number of legitimate URLs in this collection are 35,300. Among all the files, 'Benign_list_big_final.csv' is the file of our interest. 

## 1.1. Phishing URLs:

Phishing URLs are collected using PhishTank and loaded into the DataFrame.

In [ ]:
# #Downloading the phishing URLs file

# CSV_URL = 'http://data.phishtank.com/data/online-valid.csv'

# with open("Dataset/" + os.path.split(CSV_URL)[1] , 'wb') as f, \
#         requests.get(CSV_URL, stream=True) as r:
#     for line in r.iter_lines():
#         f.write(line+'\n'.encode())

In [ ]:
path = "Dataset//online-valid.csv"
df_phishing = pd.read_csv(path)

In [ ]:
df_phishing.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7380286,https://gastroutes.com/chss.imc/pnnc.php,http://www.phishtank.com/phish_detail.php?phis...,2021-12-10T06:01:05+00:00,yes,2021-12-10T06:15:42+00:00,yes,ABSA Bank
1,7380283,http://www.mygotopoolguy.com/chss.imc/pnnc.php,http://www.phishtank.com/phish_detail.php?phis...,2021-12-10T06:00:58+00:00,yes,2021-12-10T06:15:42+00:00,yes,ABSA Bank
2,7380251,https://www.correios-cttenviar-encomenda.com/P...,http://www.phishtank.com/phish_detail.php?phis...,2021-12-10T03:33:15+00:00,yes,2021-12-10T04:41:02+00:00,yes,Other
3,7380245,https://jp-mesaori-ojpbk.com/,http://www.phishtank.com/phish_detail.php?phis...,2021-12-10T02:55:27+00:00,yes,2021-12-10T04:04:53+00:00,yes,Other
4,7380233,https://www.jeifjsjejw.xyz/,http://www.phishtank.com/phish_detail.php?phis...,2021-12-10T01:46:07+00:00,yes,2021-12-10T01:52:02+00:00,yes,Other


In [ ]:
df_phishing.isnull().sum()

phish_id             0
url                  0
phish_detail_url     0
submission_time      0
verified             0
verification_time    0
online               0
target               0
dtype: int64

In [ ]:
df_phishing.shape

(11487, 8)

We would pick up 5000 samples from the above dataframe randomly.

In [ ]:
#Collecting 5,000 Phishing URLs randomly

df_phishing_final = df_phishing.sample(n = 5000, random_state = 12).copy()
df_phishing_final = df_phishing_final.reset_index(drop=True)
df_phishing_final.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7236797,http://meriprocaseinbanfro.42web.io/?i=1,http://www.phishtank.com/phish_detail.php?phis...,2021-07-21T02:13:31+00:00,yes,2021-07-22T11:07:20+00:00,yes,Other
1,6557140,http://betasus111.blogspot.com,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T23:57:08+00:00,yes,2020-05-10T00:11:42+00:00,yes,Other
2,7362306,http://roygijvhluozwnflsypmewrstt-dot-gl443933...,http://www.phishtank.com/phish_detail.php?phis...,2021-11-24T15:01:10+00:00,yes,2021-11-24T15:11:17+00:00,yes,Other
3,7273596,https://appurl.io/k8Hy2cVo8X,http://www.phishtank.com/phish_detail.php?phis...,2021-08-25T17:28:13+00:00,yes,2021-08-25T17:33:38+00:00,yes,Other
4,7161334,https://richardbashara.com/secure/login/paypal,http://www.phishtank.com/phish_detail.php?phis...,2021-06-07T15:02:10+00:00,yes,2021-06-07T15:05:14+00:00,yes,Other


In [ ]:
df_phishing_final.shape

(5000, 8)

## 1.2. Legitimate URLs:

In [ ]:
path = "Dataset//Benign_list_big_final.csv"
df_legitimate = pd.read_csv(path)
df_legitimate.columns = ["URLs"]

In [ ]:
df_legitimate.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [ ]:
#Collecting 5,000 Legitimate URLs randomly

df_legitimate_final = df_legitimate.sample(n = 5000, random_state = 12).copy()
df_legitimate_final = df_legitimate_final.reset_index(drop=True)
df_legitimate_final.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [ ]:
df_legitimate_final.shape

(5000, 1)

# 2. Feature Extraction:

In this step, features are extracted from the Legitimate URLs dataset.

The extracted features are categorized into;

1. Address Bar based Features
2. Domain based Features
3. HTML & Javascript based Features

## 2.1. Address Bar based Features:

Following features are considered as address basr based features;

- Domain of URL
- IP Address in URL
- "@" Symbol in URL
- Length of URL
- Depth of URL
- Redirection "//" in URL
- "http/https" in Domain name
- Using URL Shortening Services “TinyURL”
- Prefix or Suffix "-" in Domain

### 2.1.1. Domain of the URL:

In [ ]:
# Extracting the domain of the URL

def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.", domain):
        domain = domain.replace("www.", "")
    return domain

### 2.1.2. IP Address in the URL:

We would check for the presence of IP address in the URL, because URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL this may be one of the feature of phishig website's URL.

Hence, If the domain part of the URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
# Returns bool(True/False), whether the URL contains IP Address or not

def checkIP(url):
    try:
        ipaddress.ip_address(url)
        return True
    except:
        return False

### 2.1.3. "@" Symbol in the URL:

We would check for the presence of "@" symbol in the URL, because URLs containing "@" symbol leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

Hence, If the "@" symbol is present in the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkAtSymbol(url):
    if ("@" in url):
        return True
    else:
        return False

### 2.1.4. Length of the URL:

Here we would compute the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. Therefore, if the length of the URL is greater than or equal 54 characters then the URL is classified as phishing otherwise legitimate.

Hence, If the length of URL >= 54, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def lengthUrl(url):
    if(len(url) >= 54):
        return 1
    else:
        return 0

### 2.1.5. Depth of the URL:

Here, we would compute the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

In [ ]:
def depthUrl(url):
    depth = 0
    urlSplit = urlparse(url).path.split('/')
    for i in range(len(urlSplit)):
        if(len(urlSplit[i]) != 0):
            depth += 1
    return depth

### 2.1.6. Redirection "//" in the URL:

We would check the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. We find that if the URL starts with “HTTP”, that means the “//” should appear in the 5th position. However, if the URL employs “HTTPS” then the “//” should appear in 6th position.

If the "//" is anywhere else in the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkRedirUrl(url):
    pos = url.rfind("//")
    if(pos > 6):
        return 1
    else:
        return 0

### 2.1.7. "http/https" in the Domain name:

We would check for the presence of "http/https" in the domain part of the URL. The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.

Hence, If the URL has "http/https" in the domain part, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkHttpDomain(url):
    domain = urlparse(url).netloc
    if("http" in domain or "https" in domain):
        return 1
    else: 
        return 0

### 2.1.8. Using URL Shortening Services “TinyURL”:

URL shortening is a method on the “World Wide Web” in which a URL may be made smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

Hence, If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
#listing atll the shortening services
all_shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [ ]:
# Checking for Shortening Services present in the URL.

def checkShortUrl(url):
    if re.search(all_shortening_services, url):
        return 1
    else:
        return 0

### 2.1.9. Prefix or Suffix "-" in the Domain:

We would check the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

Hence, If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkPrefSuff(url):
    if "-" in urlparse(url).netloc:
        return 1
    else:
        return 0

## 2.2. Domain based Features:

The extracted features are categorized into;

1. DNS Record
2. Website Traffic
3. Age of Domain
3. End Period of Domain

In [ ]:
!pip install python-whois

     |████████████████████████████████| 91 kB 4.2 MB/s 
  Created wheel for python-whois: filename=python_whois-0.7.3-py3-none-any.whl size=87721 sha256=d6e5f611e792390747661782db706d1cead324aaa583aa40219ecde0e1625418
  Stored in directory: /root/.cache/pip/wheels/11/05/f7/895ce5a73665f77c8274a7d55e34fb3e6b4abbb9a7637e215b
Successfully built python-whois


### 2.2.1. DNS Record:

For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname. If the DNS record is empty or not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

### 2.2.2. Website Traffic:

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

If the rank of the domain < 100000, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [ ]:
def checkWebTraffic(url):
  try:
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank < 100000:
    return 1
  else:
    return 0

### 2.2.3. Age of Domain:

This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but different between creation and expiration time.

If age of domain > 12 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [ ]:
def checkDomainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      return 1
    else:
      return 0

### 2.2.4. End Period of Domain:

This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or less for this project.

If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [ ]:
def checkDomainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      return 0
    else:
      return 1

## 2.3. HTML and JavaScript based Features:

The extracted features are categorized into;
- IFrame Redirection
- Status Bar Customization
- Disabling Right Click
- Website Forwarding

### 2.3.1. IFrame Redirection:

IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation.

If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkIframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

### 2.3.2. Status Bar Customization:

Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the “onMouseOver” event, and check if it makes any changes on the status bar

If the response is empty or onmouseover is found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkMouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

### 2.3.3. Disabling Right Click:

Phishers use JavaScript to disable the right-click function, so that users cannot view and save the webpage source code. This feature is treated exactly as “Using onMouseOver to hide the Link”. Nonetheless, for this feature, we will search for event “event.button==2” in the webpage source code and check if the right click is disabled.

If the response is empty or onmouseover is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [ ]:
def checkRightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

### 2.3.4. Website Forwarding:

The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times.

In [ ]:
def checkForwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

# 3. Extracting URL Features:

We would create a list and a function that calls the above defined functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list accordingly.

In [ ]:
def featureExtraction(url, label):
#     print(url)
    featuresList = []
 #  Address Bar based Features
    featuresList.append(getDomain(url))
    featuresList.append(checkIP(url))
    featuresList.append(checkAtSymbol(url))
    featuresList.append(lengthUrl(url))
    featuresList.append(depthUrl(url))
    featuresList.append(checkRedirUrl(url))
    featuresList.append(checkHttpDomain(url))
    featuresList.append(checkShortUrl(url))
    featuresList.append(checkPrefSuff(url))
    
 #  Domain based Features
    dns = 0
    domainName = ""
    try:
        domainName = whois.whois(urlparse(url).netloc)
 #         print(domainName)
    except:
        dns = 1
    
    featuresList.append(dns)
    featuresList.append(checkWebTraffic(url))
    featuresList.append(1 if dns == 1 else checkDomainAge(domainName))
    featuresList.append(1 if dns == 1 else checkDomainEnd(domainName))
  
 #   HTML and JavaScript based Features
    try:
        response = requests.get(url, timeout=5)
    except:
        response = ""
        
    featuresList.append(checkIframe(response))
    featuresList.append(checkMouseOver(response))
    featuresList.append(checkRightClick(response))
    featuresList.append(checkForwarding(response))
    featuresList.append(label)
    
    return featuresList

# 4. Legitimate URLs:

In [ ]:
df_legitimate_final.shape

(5000, 1)

In [ ]:
# legitimateFeatures = []
# label = 0

# for i in range(0, 5000):
#     print(i)
#     url = df_legitimate_final['URLs'][i]
#     legitimateFeatures.append(featureExtraction(url, label))

In [ ]:
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

df_legitimate_final = pd.DataFrame(legitimateFeatures, columns= feature_names)
df_legitimate_final.isnull().sum()

Domain           0
Have_IP          0
Have_At          0
URL_Length       0
URL_Depth        0
Redirection      0
https_Domain     0
TinyURL          0
Prefix/Suffix    0
DNS_Record       0
Web_Traffic      0
Domain_Age       0
Domain_End       0
iFrame           0
Mouse_Over       0
Right_Click      0
Web_Forwards     0
Label            0
dtype: int64

In [ ]:
# # Storing the extracted legitimate URLs features to csv file
# df_legitimate_final.to_csv('Dataset//legitimate-urls.csv', index= False)

In [ ]:
# phishingFeatures = []
# label = 1

# for i in range(0, 5000):
#     print(i)
#     url = df_phishing_final['url'][i]
#     phishingFeatures.append(featureExtraction(url, label))

In [ ]:
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

df_phishing_final = pd.DataFrame(phishingFeatures, columns= feature_names)
df_phishing_final.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,meriprocaseinbanfro.42web.io,False,False,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1
1,betasus111.blogspot.com,False,False,0,0,0,0,1,0,1,1,1,1,0,0,1,0,1
2,roygijvhluozwnflsypmewrstt-dot-gl44393333333.r...,False,False,1,1,0,0,1,1,1,1,1,1,0,0,1,0,1
3,appurl.io,False,False,0,1,0,0,0,0,1,0,1,1,0,0,1,0,1
4,richardbashara.com,False,False,0,3,0,0,0,0,1,1,1,1,0,0,1,0,1


In [ ]:
# # Storing the extracted phishing URLs features to csv file
# df_phishing_final.to_csv('Dataset//phishing-urls.csv', index= False)

# 5. Final Dataset:

Here, we would combine legitimate and phishing URLs into a single csv file for the Machine Learning training.

In [5]:
legitimate = pd.read_csv("Dataset//legitimate-urls.csv")
phishing = pd.read_csv("Dataset//phishing-urls.csv")

In [8]:
#Concatenating the dataframes into one 

leg_phish_data = pd.concat([legitimate, phishing]).reset_index(drop=True)
leg_phish_data.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,graphicriver.net,False,False,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0
1,ecnavi.jp,False,False,1,1,1,0,0,0,1,1,1,1,0,0,1,0,0
2,hubpages.com,False,False,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0
3,extratorrent.cc,False,False,1,3,0,0,0,0,1,0,1,1,0,0,1,0,0
4,icicibank.com,False,False,1,3,0,0,0,0,1,1,1,1,0,0,1,0,0


In [9]:
leg_phish_data.shape

(10000, 18)

In [11]:
# Storing the data in CSV file

leg_phish_data.to_csv('Dataset//leg_phish-urls.csv', index=False)

# 6. Refrences

- https://archive.ics.uci.edu/ml/datasets/Phishing+Websites
- https://github.com/shreyagopal/Phishing-Website-Detection-by-Machine-Learning-Techniques/blob/master/URL%20Feature%20Extraction.ipynb